In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

obss_intern_competition_2025_path = kagglehub.competition_download('obss-intern-competition-2025')
umutkapukran_obss_dataset1_path = kagglehub.dataset_download('umutkapukran/obss-dataset1')

print('Data source import complete.')


100%|██████████| 995M/995M [00:45<00:00, 22.8MB/s]

Extracting files...


100%|██████████| 995M/995M [00:47<00:00, 22.1MB/s]

Extracting files...


Data source import complete.


In [4]:
import pandas as pd

train_df = pd.read_csv(f"{umutkapukran_obss_dataset1_path}/train.csv")
test_df = pd.read_csv(f"{umutkapukran_obss_dataset1_path}/test.csv")


In [5]:
train_image_dir = f"{umutkapukran_obss_dataset1_path}/train/train"
test_image_dir = f"{umutkapukran_obss_dataset1_path}/test/test"


In [6]:
# 2. image_id kolonunu temizle
train_df["image_id"] = train_df["image_id"].astype(str)
train_df["image_id"] = train_df["image_id"].str.replace(".jpg", "", regex=False)
train_df["image_id"] = train_df["image_id"] + ".jpg"

test_df["image_id"] = test_df["image_id"].astype(str)
test_df["image_id"] = test_df["image_id"].str.replace(".jpg", "", regex=False)
test_df["image_id"] = test_df["image_id"] + ".jpg"


In [7]:
!pip install transformers datasets sentence-transformers accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.7 MB/s eta 0:00:00


In [8]:
import torch
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

# BLIP modelini ve işlemcisini yükle
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [9]:
class CaptionDataset(Dataset):
    def __init__(self, dataframe, image_dir, processor):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.image_dir, row["image_id"])
        image = Image.open(image_path).convert("RGB")
        caption = row["caption"]

        inputs = self.processor(images=image, text=caption, return_tensors="pt", padding="max_length", truncation=True, max_length=32)
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = inputs["input_ids"]
        return inputs


In [10]:
from torch.utils.data import DataLoader, random_split
from transformers import TrainingArguments, Trainer, default_data_collator

# Dataset
train_dataset = CaptionDataset(train_df, train_image_dir, processor)

# %90 eğitim, %10 validation
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_ds, val_ds = random_split(train_dataset, [train_size, val_size])

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="./blip-captioning",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=100,
    report_to="none",  # wandb, tensorboard gibi servisleri devre dışı bırakır
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor,
    data_collator=default_data_collator,
)


<ipython-input-10-053d27108249>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

Step,Training Loss
100,3.708800
200,2.240600
300,2.059900
400,1.985300
500,1.963200
600,1.907000
700,1.840700
800,1.896400
900,1.873800
1000,1.753100


TrainOutput(global_step=14424, training_loss=1.1322287235138355, metrics={'train_runtime': 6711.5623, 'train_samples_per_second': 8.596, 'train_steps_per_second': 2.149, 'total_flos': 3.4234610496861635e+19, 'train_loss': 1.1322287235138355, 'epoch': 3.0})

In [12]:
from tqdm import tqdm
from PIL import Image
import os
import pandas as pd

captions = []

model.eval()
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = os.path.join(test_image_dir, row["image_id"])
    image = Image.open(image_path).convert("RGB")

    # Görseli modele uygun şekilde işle
    inputs = processor(images=image, return_tensors="pt").to(device)

    # Caption üret
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_length=32)
    caption = processor.decode(output_ids[0], skip_special_tokens=True)

    # Listeye ekle
    captions.append({
        "image_id": row["image_id"],
        "caption": caption
    })

# DataFrame'e çevir
submission_df = pd.DataFrame(captions)

# CSV olarak kaydet
submission_df.to_csv("submission.csv", index=False)


100%|██████████| 3771/3771 [29:18<00:00,  2.14it/s]


In [16]:
submission_df["image_id"] = submission_df["image_id"].str.replace(".jpg", "", regex=False)

In [19]:
submission_df.to_csv("submission1.csv", index=False)


In [20]:
from google.colab import files
files.download("submission1.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>